In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_log_error # set squared=False

## Functions

In [3]:
def preliminary_EDA(df):
    # Show first rows
    display(df.head())
    
    # Are there missing values?
    display(df.isna().sum())
    
    # Summary of data
    display(df.describe())
    
    # data types
    display(df.dtypes)

## Import data - Initial EDA

In [4]:
train_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
preliminary_EDA(train_df)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

,id,store_nbr,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2.750000e+01,3.577757e+02,2.602770e+00
std,8.662819e+05,1.558579e+01,1.101998e+03,1.221888e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,5.400000e+01,1.247170e+05,7.410000e+02


id               int64
date            object
store_nbr        int64
family          object
sales          float64
onpromotion      int64
dtype: object

In [5]:
train_df['timestamp']=pd.to_datetime(train_df.date)
train_df.drop('date', axis=1, inplace=True)
train_df.head()
train_df.dtypes

id                      int64
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
timestamp      datetime64[ns]
dtype: object

In [6]:
oil_prices_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
preliminary_EDA(oil_prices_df)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


date           0
dcoilwtico    43
dtype: int64

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


date           object
dcoilwtico    float64
dtype: object

In [7]:
oil_prices_df['timestamp']=pd.to_datetime(oil_prices_df.date)
oil_prices_df.drop('date', axis=1, inplace=True)

oil_prices_df.head()
oil_prices_df.dtypes

dcoilwtico           float64
timestamp     datetime64[ns]
dtype: object

In [8]:
# Missing oil prices - investigate further
missing_oil_prices_df = oil_prices_df[oil_prices_df.dcoilwtico.isna()]
missing_oil_prices_df

,dcoilwtico,timestamp
0,NaN,2013-01-01
14,NaN,2013-01-21
34,NaN,2013-02-18
63,NaN,2013-03-29
104,NaN,2013-05-27
132,NaN,2013-07-04
174,NaN,2013-09-02
237,NaN,2013-11-28
256,NaN,2013-12-25
261,NaN,2014-01-01


These dates seem to be similar through the years - potentially holidays when the market is closed. Will investigate further after merging the datasets

In [9]:
holidays_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
preliminary_EDA(holidays_df)

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

,date,type,locale,locale_name,description,transferred
count,350,350,350,350,350,350
unique,312,6,3,24,103,2
top,2014-06-25,Holiday,National,Ecuador,Carnaval,False
freq,4,221,174,174,10,338


date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object

In [10]:
holidays_df['timestamp']=pd.to_datetime(holidays_df.date)
holidays_df.rename(columns={'type': 'special_day_type'}, inplace=True)
holidays_df.drop('date', axis=1, inplace=True)
holidays_df.head()
holidays_df.dtypes

special_day_type            object
locale                      object
locale_name                 object
description                 object
transferred                   bool
timestamp           datetime64[ns]
dtype: object

In [11]:
missing_oil_prices_df = pd.merge(missing_oil_prices_df, holidays_df, how='left', on='timestamp')
missing_oil_prices_df

,dcoilwtico,timestamp,special_day_type,locale,locale_name,description,transferred
0,NaN,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,NaN,2013-01-21,NaN,NaN,NaN,NaN,NaN
2,NaN,2013-02-18,NaN,NaN,NaN,NaN,NaN
3,NaN,2013-03-29,NaN,NaN,NaN,NaN,NaN
4,NaN,2013-05-27,NaN,NaN,NaN,NaN,NaN
5,NaN,2013-07-04,NaN,NaN,NaN,NaN,NaN
6,NaN,2013-09-02,NaN,NaN,NaN,NaN,NaN
7,NaN,2013-11-28,NaN,NaN,NaN,NaN,NaN
8,NaN,2013-12-25,Holiday,National,Ecuador,Navidad,False
9,NaN,2014-01-01,Holiday,National,Ecuador,Primer dia del ano,False


It looks like the days oil prices are missing do not always coincide with holidays. Conisdering the prices are only missing for a 45 days in total, I can impute these values with the previous day's prices. I will keep track of the imputed rows

In [12]:
oil_prices_df['oil_price_imputed'] = oil_prices_df.apply(lambda x: pd.isna(x.dcoilwtico), axis=1)
oil_prices_df.dcoilwtico = oil_prices_df.dcoilwtico.bfill()
oil_prices_df.head()

,dcoilwtico,timestamp,oil_price_imputed
0,93.14,2013-01-01,True
1,93.14,2013-01-02,False
2,92.97,2013-01-03,False
3,93.12,2013-01-04,False
4,93.20,2013-01-07,False


In [13]:
stores_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
preliminary_EDA(stores_df)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

,store_nbr,cluster
count,54.000000,54.000000
mean,27.500000,8.481481
std,15.732133,4.693395
min,1.000000,1.000000
25%,14.250000,4.000000
50%,27.500000,8.500000
75%,40.750000,13.000000
max,54.000000,17.000000


store_nbr     int64
city         object
state        object
type         object
cluster       int64
dtype: object

In [14]:
stores_df.rename(columns={'type': 'store_type'}, inplace=True)
stores_df.head()

,store_nbr,city,state,store_type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [15]:
transactions_df = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
preliminary_EDA(transactions_df)

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


date            0
store_nbr       0
transactions    0
dtype: int64

,store_nbr,transactions
count,83488.000000,83488.000000
mean,26.939237,1694.602158
std,15.608204,963.286644
min,1.000000,5.000000
25%,13.000000,1046.000000
50%,27.000000,1393.000000
75%,40.000000,2079.000000
max,54.000000,8359.000000


date            object
store_nbr        int64
transactions     int64
dtype: object

In [16]:
transactions_df['timestamp']=pd.to_datetime(transactions_df.date)
transactions_df.drop('date', axis=1, inplace=True)

transactions_df.head()
transactions_df.dtypes

store_nbr                int64
transactions             int64
timestamp       datetime64[ns]
dtype: object

## Merge datasets

In [17]:
df = pd.merge(train_df, oil_prices_df, on='timestamp', how='left')
del train_df, oil_prices_df
df = pd.merge(df, holidays_df, on='timestamp', how='left')
del holidays_df
df = pd.merge(df, transactions_df, on=['timestamp', 'store_nbr'], how='left')
del transactions_df
df = pd.merge(df, stores_df, on='store_nbr', how='left')
del stores_df

df.head()

,id,store_nbr,family,sales,onpromotion,timestamp,dcoilwtico,oil_price_imputed,special_day_type,locale,locale_name,description,transferred,transactions,city,state,store_type,cluster
0,0,1,AUTOMOTIVE,0.0,0,2013-01-01,93.14,True,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13
1,1,1,BABY CARE,0.0,0,2013-01-01,93.14,True,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13
2,2,1,BEAUTY,0.0,0,2013-01-01,93.14,True,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13
3,3,1,BEVERAGES,0.0,0,2013-01-01,93.14,True,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13
4,4,1,BOOKS,0.0,0,2013-01-01,93.14,True,Holiday,National,Ecuador,Primer dia del ano,False,NaN,Quito,Pichincha,D,13


In [18]:
df['month'] = df.timestamp.dt.month
df['day'] = df.timestamp.dt.day
df['year'] = df.timestamp.dt.year
df['day_of_week'] = df.timestamp.dt.day_of_week
df.head()

,id,store_nbr,family,sales,onpromotion,timestamp,dcoilwtico,oil_price_imputed,special_day_type,locale,...,transferred,transactions,city,state,store_type,cluster,month,day,year,day_of_week
0,0,1,AUTOMOTIVE,0.0,0,2013-01-01,93.14,True,Holiday,National,...,False,NaN,Quito,Pichincha,D,13,1,1,2013,1
1,1,1,BABY CARE,0.0,0,2013-01-01,93.14,True,Holiday,National,...,False,NaN,Quito,Pichincha,D,13,1,1,2013,1
2,2,1,BEAUTY,0.0,0,2013-01-01,93.14,True,Holiday,National,...,False,NaN,Quito,Pichincha,D,13,1,1,2013,1
3,3,1,BEVERAGES,0.0,0,2013-01-01,93.14,True,Holiday,National,...,False,NaN,Quito,Pichincha,D,13,1,1,2013,1
4,4,1,BOOKS,0.0,0,2013-01-01,93.14,True,Holiday,National,...,False,NaN,Quito,Pichincha,D,13,1,1,2013,1
